In [ ]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
import os
import awpy
from demoparser2 import DemoParser
from awpy import Demo
from pathlib import Path
from awpy.stats import adr
from awpy.stats import kast
from awpy.stats import rating
from awpy.stats import calculate_trades

#folder_path = r'C:\Users\bayli\Documents\CS Demos\IEM_Katowice_2025'
folder_path = r'C:\Users\bayli\Documents\Git Projects\test_demos'
file_path = r'C:\Users\bayli\Documents\Git Projects\test_demos\natus-vincere-vs-faze-m1-inferno.dem'

# Creating DataFrames
df_flashes = pd.DataFrame()
df_he = pd.DataFrame()
df_infernos = pd.DataFrame()
df_smoke = pd.DataFrame()
df_kills = pd.DataFrame()
df_rounds = pd.DataFrame()
df_all_first_kills = pd.DataFrame()
df_adr = pd.DataFrame()
df_kast = pd.DataFrame()
team_rounds_won = pd.DataFrame()
players_id = pd.DataFrame()

df_files = pd.DataFrame(columns=['file_id', 'file_name'])
i = 1
file_id_counter = 1

In [2]:
def add_round_winners(ticks_df, rounds_df):
    ticks_df = ticks_df.to_pandas()
    rounds_df = rounds_df.to_pandas()
    
    # Changes the first round losing steak to 0
    # This is necessary because the first round is not a losing streak
    ticks_df.loc[ticks_df['round_num'] == 1, ['ct_losing_streak', 't_losing_streak']] = 0

    # Makes sure the columns exists
    rounds_df['ct_team_clan_name'] = None
    rounds_df['t_team_clan_name'] = None
    rounds_df['winner_clan_name'] = None
    rounds_df['CT_team_current_equip_value'] = None
    rounds_df['T_team_current_equip_value'] = None
    rounds_df['ct_losing_streak'] = None
    rounds_df['t_losing_streak'] = None

    for idx, row in rounds_df.iterrows():
        freeze_end_tick = row['freeze_end']
        winner = row['winner']

        # Takes all corresponding entries
        first_tick_df = ticks_df[ticks_df['tick'] == freeze_end_tick]

        # Takes the name for every team
        try:
            CT_team = first_tick_df[first_tick_df['side'] == 'ct']['team_clan_name'].iloc[0]
        except IndexError:
            CT_team = None
        
        try:
            T_team = first_tick_df[first_tick_df['side'] == 't']['team_clan_name'].iloc[0]
        except IndexError:
            T_team = None

        # Takes the current equip value for every team
        try:
            CT_team_current_equip_value = first_tick_df[first_tick_df['side'] == 'ct']['current_equip_value'].sum()
        except KeyError:
            CT_team_current_equip_value = None

        try:
            T_team_current_equip_value = first_tick_df[first_tick_df['side'] == 't']['current_equip_value'].sum()
        except KeyError:
            T_team_current_equip_value = None

        # Takes the losing streak for every team
        try:
            ct_losing_streak = first_tick_df[first_tick_df['side'] == 'ct']['ct_losing_streak'].iloc[0]
        except IndexError:
            ct_losing_streak = None

        try:
            t_losing_streak = first_tick_df[first_tick_df['side'] == 't']['t_losing_streak'].iloc[0]
        except IndexError:
            t_losing_streak = None

        if winner == 'ct':
            winner_clan = CT_team
        elif winner in ['t', 'TERRORIST']:
            winner_clan = T_team
        else:
            winner_clan = None
            print(f"[!] Round {idx} - winner error: '{winner}'")
            
        # Fill Columns in the DataFrame
        rounds_df.at[idx, 'ct_team_clan_name'] = CT_team
        rounds_df.at[idx, 't_team_clan_name'] = T_team
        rounds_df.at[idx, 'winner_clan_name'] = winner_clan
        rounds_df.at[idx, 'CT_team_current_equip_value'] = CT_team_current_equip_value
        rounds_df.at[idx, 'T_team_current_equip_value'] = T_team_current_equip_value
        rounds_df.at[idx, 'ct_losing_streak'] = ct_losing_streak
        rounds_df.at[idx, 't_losing_streak'] = t_losing_streak


    return rounds_df

In [1]:
def add_buy_type(row):

    if row['round_num'] in [1, 13]:
        return "Pistol", "Pistol"

    if row['CT_team_current_equip_value'] < 5000:
        ct_buy_type = "Full Eco"
    elif 5000 <= row['CT_team_current_equip_value'] < 10000:
        ct_buy_type = "Semi-Eco"
    elif 10000 <= row['CT_team_current_equip_value'] < 20000:
        ct_buy_type = "Semi-Buy"
    elif row['CT_team_current_equip_value'] >= 20000:
        ct_buy_type = "Full Buy"
    else:
        ct_buy_type = "Unknown"

    if row['T_team_current_equip_value'] < 5000:
        t_buy_type = "Full Eco"
    elif 5000 <= row['T_team_current_equip_value'] < 10000:
        t_buy_type = "Semi-Eco"
    elif 10000 <= row['T_team_current_equip_value'] < 20000:
        t_buy_type = "Semi-Buy"
    elif row['T_team_current_equip_value'] >= 20000:
        t_buy_type = "Full Buy"
    else:
        t_buy_type = "Unknown"

    return ct_buy_type, t_buy_type

In [4]:
def calculate_5v4_advantage(rounds_df, first_kills_df):

    # Makes sure the columns exists
    rounds_df['5v4_advantage'] = None

    # Checks what team got the first kill
    for idx, row in rounds_df.iterrows():
        round_num = row['round_num']

        # Filters the first kills DataFrame for the current round
        first_kill = first_kills_df[first_kills_df['round_num'] == round_num]

        if not first_kill.empty:
            # Gets the team that made the first kill
            killer_team = first_kill.iloc[0]['attacker_side']

            # Defines the advantage based on the killer team
            if killer_team == 'ct':
                rounds_df.at[idx, '5v4_advantage'] = 'ct'
            elif killer_team == 't':
                rounds_df.at[idx, '5v4_advantage'] = 't'

    return rounds_df

In [35]:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.dem'):

        file_path = os.path.join(folder_path, file_name)
        dem = Demo(file_path)
        dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak", "is_terrorist_timeout", "is_ct_timeout", "4k_rounds_total"])
        
        this_file_df_ticks = dem.ticks
        this_file_df_rounds = dem.rounds
        
        this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)
        this_file_df_rounds[['CT_buy_type', 'T_buy_type']] = this_file_df_rounds.apply(add_buy_type, axis=1, result_type='expand')

        # Opening Kills Data
        this_file_df_kills = awpy.stats.calculate_trades(demo=dem)
        this_file_df_kills = this_file_df_kills.with_columns(
            this_file_df_kills['attacker_steamid'].cast(pl.Utf8),
            this_file_df_kills['assister_steamid'].cast(pl.Utf8),
            this_file_df_kills['victim_steamid'].cast(pl.Utf8)
        )
        this_file_df_kills = this_file_df_kills.to_pandas()
        first_kills = this_file_df_kills.sort_values(by=['round_num', 'tick'])
        first_kills = first_kills.groupby('round_num').first().reset_index()
        df_all_first_kills = pd.concat([df_all_first_kills, first_kills], ignore_index=True)   

        this_file_df_rounds = calculate_5v4_advantage(this_file_df_rounds, df_all_first_kills)
        this_file_df_rounds['demo_file_name'] = file_name   

        df_files = pd.concat([df_files, pd.DataFrame({'file_id': [file_id_counter], 'file_name': [file_name]})], ignore_index=True)
        file_id_counter += 1
          

In [ ]:
# Listando entradas únicas nas colunas
unique_ct_timeout = df_ticks['is_ct_timeout'].unique()
unique_terrorist_timeout = df_ticks['is_terrorist_timeout'].unique()

# Exibindo os resultados
print("Entradas únicas em is_ct_timeout:", unique_ct_timeout)
print("Entradas únicas em is_terrorist_timeout:", unique_terrorist_timeout)

In [ ]:
dem = Demo(file_path)
dem.parse(player_props=["team_clan_name","total_rounds_played", "current_equip_value", "ct_losing_streak", "t_losing_streak", "is_terrorist_timeout", "is_ct_timeout", "4k_rounds_total"])

this_file_df_ticks = dem.ticks
this_file_df_rounds = dem.rounds
this_file_df_rounds = add_round_winners(this_file_df_ticks,this_file_df_rounds)

In [ ]:
this_file_df_rounds